# Neural Network Models for Speaker Classification

Last Update: 12/01/2019 by Chen Liang

This notebook includes the following deep learning models: 1) CNN on MFCC 2) CNN on Chroma

In [68]:
import librosa
import numpy as np
import shutil
import os
from joblib import Parallel, delayed
import multiprocessing 
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [42]:
raw_dir='recordings/selected_recordings/'
silence_removed_dir='recordings/silence_removed/'
unified_dir='recordings/unified/'

## Preparation: Data Understanding

For this dataset, each speaker is required to read the following paragraph:
>*Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.*

Which has the following phonemes

## Convolutional Neural Network on MFCC

Key idea: Use filters to extract key features from a series of adjacent MFCC slices

In [233]:
y,sr=librosa.load(unified_dir+'spanish6.mp3.wav')
librosa.feature.chroma_stft(y=y, sr=sr).shape

(12, 861)

In [41]:
y_stretched[:sr]

array([0.02146304, 0.01559818, 0.0085238 , ..., 0.05236012, 0.05111545,
       0.04814873], dtype=float32)

In [31]:
y,sr=librosa.load(unified_dir+'spanish12.mp3.wav')
print(sr)
print(librosa.get_duration(y=y,sr=sr))
result1=librosa.feature.mfcc(y=y,hop_length=int(sr/100), n_fft=int(sr/40))

22050
24.938231292517006


In [32]:
y,sr=librosa.load(unified_dir+'spanish6.mp3.wav')
print(librosa.get_duration(y=y,sr=sr))
result2=librosa.feature.mfcc(y=y,hop_length=int(sr/100), n_fft=int(sr/40))

21.687437641723356


In [39]:
result1.shape

(20, 2055)

In [34]:
result2.shape

(20, 2174)

In [43]:
y,sr=librosa.load(unified_dir+'spanish12.mp3.wav')
print(sr)
print(librosa.get_duration(y=y,sr=sr))
result1=librosa.feature.mfcc(y=y,hop_length=int(sr/100), n_fft=int(sr/40))
result1=librosa.feature.mfcc(y=y,hop_length=int(sr/100), n_fft=int(sr/40))

22050
19.989977324263037


### Extract MFCC

In [46]:
result1.shape

(20, 2004)

In [61]:
mfcc_window_size=0.025
mfcc_stride_size=0.01
mfcc_num_of_features=14

In [62]:
def extract_mfcc(fname):
    y,sr=librosa.load(unified_dir+fname)
    result=librosa.feature.mfcc(y=y,n_mfcc=mfcc_num_of_features,hop_length=int(mfcc_window_size*sr), n_fft=int(mfcc_stride_size*sr))
    result=librosa.util.normalize(result)
    #Add label as well
    if 'mandarin' in fname:
        label=0
    elif 'english' in fname:
        label=1
    elif 'spanish' in fname:
        label=2
    else:
        label=-1
    return [result,label]

In [63]:
result=Parallel(n_jobs=12)(delayed(extract_mfcc)(fname) for fname in os.listdir(unified_dir))

In [177]:
X=[]
y=[]
for i in result:
    X.append(i[0])
    y.append(i[1])

### Set up CNN Model

<b> Model 1:</b> Directly fit on extracted MFCC features. Input shape is (14,800)

In [166]:
#input_shape=X_all[:,:,0].shape
input_shape=(14,800,1)
print(input_shape)

(14, 800, 1)


In [178]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.1)
X_train=np.expand_dims(X_train,axis=3)
X_test=np.expand_dims(X_test,axis=3)

In [179]:
def change_input_format(X):
    Xc=np.stack(X,axis=0)
#     Xc=np.swapaxes(Xc,0,1)
#     Xc=np.swapaxes(Xc,1,2)
    return Xc

In [180]:
X_train=change_input_format(X_train)
X_test=change_input_format(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [182]:
X_test.shape

(60, 14, 800, 1)

In [185]:
y_train.shape

(540,)

In [199]:
def create_model1():
    model1=models.Sequential()
    model1.add(layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=input_shape))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Flatten())
    model1.add(layers.Dense(64, activation='relu'))
    model1.add(layers.Dense(3, activation='softmax'))
    return model1

In [200]:
model1=create_model1()
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model1.fit(X_train, y_train, epochs=50, 
                    validation_data=(X_test, y_test))

Train on 540 samples, validate on 60 samples
Epoch 1/50
540/540 [==============================] - 2s 3ms/sample - loss: 0.9374 - accuracy: 0.5889 - val_loss: 0.8787 - val_accuracy: 0.6167
Epoch 2/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.8885 - accuracy: 0.6222 - val_loss: 0.8546 - val_accuracy: 0.6167
Epoch 3/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.9027 - accuracy: 0.6222 - val_loss: 0.8421 - val_accuracy: 0.6167
Epoch 4/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.8604 - accuracy: 0.6222 - val_loss: 0.8221 - val_accuracy: 0.6167
Epoch 5/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.8049 - accuracy: 0.6222 - val_loss: 0.7467 - val_accuracy: 0.6167
Epoch 6/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.7735 - accuracy: 0.6759 - val_loss: 0.7194 - val_accuracy: 0.6167
Epoch 7/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.6477 - accura

In [225]:
def create_model2():
    model1=models.Sequential()
    model1.add(layers.Conv2D(32,kernel_size=(5,5),strides=(1,1),activation='relu',input_shape=input_shape))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model1.add(layers.MaxPooling2D((2, 2)))
    model1.add(layers.Flatten())
    model1.add(layers.Dense(64, activation='relu'))
    model1.add(layers.Dense(3, activation='softmax'))
    return model1

In [229]:
print(y_test)

[0 1 2 2 0 1 1 1 1 1 1 2 1 1 1 2 1 2 2 1 1 1 2 2 2 1 1 1 2 2 1 2 2 1 1 2 2
 0 2 0 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 0 1 1 2 1]


In [226]:
model2=create_model2()
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model2.fit(X_train, y_train, epochs=50, 
                    validation_data=(X_test, y_test))

Train on 540 samples, validate on 60 samples
Epoch 1/50
540/540 [==============================] - 1s 2ms/sample - loss: 0.9190 - accuracy: 0.6148 - val_loss: 0.8633 - val_accuracy: 0.6167
Epoch 2/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.8756 - accuracy: 0.6222 - val_loss: 0.8122 - val_accuracy: 0.6167
Epoch 3/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.8373 - accuracy: 0.6352 - val_loss: 0.7953 - val_accuracy: 0.7333
Epoch 4/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.8251 - accuracy: 0.6852 - val_loss: 0.7375 - val_accuracy: 0.75000s - loss: 0.8295 - accuracy: 0.68
Epoch 5/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.7279 - accuracy: 0.7074 - val_loss: 0.6229 - val_accuracy: 0.8000
Epoch 6/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.6271 - accuracy: 0.7352 - val_loss: 0.5757 - val_accuracy: 0.7000
Epoch 7/50
540/540 [==============================] - 1s 

In [227]:
def create_model3():
    model1=models.Sequential()
    model1.add(layers.Conv2D(32,kernel_size=(14,5),strides=(1,1),activation='relu',input_shape=input_shape))
    model1.add(layers.MaxPooling2D((1, 2)))
    model1.add(layers.Flatten())
    model1.add(layers.Dense(32, activation='relu'))
    model1.add(layers.Dense(3, activation='softmax'))
    return model1

In [228]:
model3=create_model3()
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model3.fit(X_train, y_train, epochs=50, 
                    validation_data=(X_test, y_test))

Train on 540 samples, validate on 60 samples
Epoch 1/50
540/540 [==============================] - 1s 1ms/sample - loss: 0.9283 - accuracy: 0.5759 - val_loss: 0.8729 - val_accuracy: 0.6167
Epoch 2/50
540/540 [==============================] - 0s 191us/sample - loss: 0.8890 - accuracy: 0.6222 - val_loss: 0.8635 - val_accuracy: 0.6167
Epoch 3/50
540/540 [==============================] - 0s 178us/sample - loss: 0.8754 - accuracy: 0.6222 - val_loss: 0.8386 - val_accuracy: 0.6167
Epoch 4/50
540/540 [==============================] - 0s 174us/sample - loss: 0.8572 - accuracy: 0.6222 - val_loss: 0.8172 - val_accuracy: 0.6167
Epoch 5/50
540/540 [==============================] - 0s 181us/sample - loss: 0.8273 - accuracy: 0.6222 - val_loss: 0.7818 - val_accuracy: 0.6167
Epoch 6/50
540/540 [==============================] - 0s 179us/sample - loss: 0.7799 - accuracy: 0.6278 - val_loss: 0.7535 - val_accuracy: 0.6167
Epoch 7/50
540/540 [==============================] - 0s 185us/sample - loss: 0.7

## Convolutional Neural Network on Chroma

### Extract Chroma

In [ ]:
def get_chroma(fname):
    y,sr=librosa.load(unified_dir+fname)
    return librosa.feature.chroma_stft(y=y, sr=sr).shape